<p id="eda" style="font-size:30px; text-align:center; font-weight:bold">BART (Bidirectional and Auto-Regressive Transformers)</p>

<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 20px">BART is a model created by Facebook AI that uses transformers to work on both the encoder and decoder sides, like how GPT-2 and BERT do. BART combines the strengths of models like BERT, which are excellent at decoding coherent sequences of text with models like GPT-2, which are excellent at comprehending the context from supplied text (encoder tasks)</p>

<p style="font-size: 20px; font-weight:bold"><u>Justification (Why used for medical chatbot (doctor-patient dialogues dataset))</u></p>

<p style="font-size: 20px">BART is a model that specialised at both comprehending and generating text. It excellent at taking details from conversations, especially because of its capacity to consider data from both the start and end of a dialogue. This ability enables it to produce accurate and relevant responses, which makes it an excellent choice for medical chatbots. Its versatility and ability to provide in-depth responses allow it to answer queries as well as write text, ensuring that patients receive information that will be precise and clear.</p>

<p style="font-size: 20px; font-weight:bold"><u>Version</u></p>

<p style="font-size: 20px">Facebook/bart-base, version has been used in this project for the training on patient-doctor dialogues. Facebook/bart-base has approximately 140 million parameters. The training time can be time consuming, but it can handle variety of tasks due its architecture</p>

<div style="width:100%;height:3px; background-color:black"></div>

<p id="lib" style="font-size:30px; text-align:center; font-weight:bold">Required libraries or packages</p> <a href="#top">Back To Top</a>

<div style="width:100%;height:3px; background-color:black"></div>

In [1]:
import torch # PyTorch open-source deep learning framework
import json # json library to load JSON data
from sklearn.model_selection import train_test_split # train_test_split method from scikit learn for datset splitting 

from transformers import (
   
    TrainingArguments, # class to store arguments for training transformers models
    Trainer,  # trainer class for training and evaluating transformers models
    TrainerCallback, # case class for callbacks in the Trainer class
    IntervalStrategy, # strategy to define when to run callbacks
    BartTokenizer, # bart tokenizer
    BartForConditionalGeneration, # bart model for text generation, specialised for text summarisation but can be used for answering questioning the problem I am solving
)
from nltk.translate.bleu_score import sentence_bleu # to calculate BLEU score
from rouge import Rouge # to calculate ROUGE score
from language_tool_python import LanguageTool# used to check grammar
import spacy # library for natural language processing
from transformers import EvalPrediction # used to store predictions and labels ids together
import torch.nn as nn # to calculate loss
torch.cuda.empty_cache() # clear cache from gpu

2023-09-06 02:53:59.106561: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 02:53:59.988175: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-09-06 02:53:59.988234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-09-06 02:53:59.988240: W tensorflow/compiler/tf2tensorrt/utils/py_utils

<div style="width:100%;height:1px; background-color:black"></div>

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # check if GPU support available than use GPU otherwise use CPU


<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Load Dataset</p>

In [3]:
with open("cleaned_medical_dialogues_dataset.json", "r") as f: # load the data from the JSON file
    data = json.load(f)

data = data[::10] # sampling 1/10th of the dataset to manage time and computational resources

print(len(data)) # print length number of samples in the sampled data

24711


<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Load Pre-trained Model</p>

In [4]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)


Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Dataset Preparation</p>

In [5]:
inputs = [item["Patient"] for item in data] # extracting the Patient queries from the data and storing them in the 'inputs' list

targets = [item["Doctor"] for item in data] # extracting the Doctor responses from the data and storing them in the 'targets' list


In [6]:
input_encodings = tokenizer(inputs, truncation=True, padding=True, return_tensors="pt") # tokenizing the inputs using the tokenizer, truncating if they exceed the model's max length, padding them to the same length, and returning them as PyTorch tensors

target_encodings = tokenizer(targets, truncation=True, padding=True, return_tensors="pt") # tokenizing the targets using the tokenizer, truncating if they exceed the model's max length, padding them to the same length, and returning them as PyTorch tensors



In [7]:
class MedicalChatDataset(torch.utils.data.Dataset): # defining a PyTorch dataset class for the medical chat data
    
    # constructor method to store tokenized input and target  encodings
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    # method to fetch an item from the dataset given an index 'idx'
    def __getitem__(self, idx):
        # constructing a dictionary that contains input IDs, attention masks, and corresponding target responses
        item = {
            "input_ids": self.input_encodings["input_ids"][idx],
            "attention_mask": self.input_encodings["attention_mask"][idx], 
            "labels": self.target_encodings["input_ids"][idx]
        }
        return item

    # Method to return the total number of items in the dataset
    def __len__(self):
        return len(self.input_encodings["input_ids"])



In [8]:
dataset = MedicalChatDataset(input_encodings, target_encodings) # calling the MedicalChatDataset class and returning the structured dataset required for the model

<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Split Dataset</p>

<p style="font-size: 20px">
  The training set and validation/test set are two sets that are typically separated from the available dataset in machine learning and deep learning.
This separation helps in evaluating the model's performance on unseen data. The main goal is to prevent the model from overfitting the training set of data. If a model performs exceptionally well on training data but poorly on validation data, it is obviously overfitted.</p>

In [9]:

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)  # 20% of data as validation


<p style="font-size: 20px">The parameter <b>test_size=0.2</b> indicates that 20% of the dataset will be used as the validation set, while the remaining 80% will be used for training.</p>

<div style="width:100%;height:1px; background-color:black"></div>

<p id="lib" style="font-size:30px; text-align:center; font-weight:bold">Fine tuning facebook/bart-base</p> <a href="#top">Back To Top</a>

<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Define Early Stopping</p>

<p style="font-size: 20px">Callback function to halt the training process when there is no improvement in the model
</p>

In [10]:
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience): # intialize the callback with number of epochs to wait
        self.early_stopping_patience = early_stopping_patience
        self.patience_counter = 0 # counter to keep track of epochs
        self.best_score = None # store the best evaluation score

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        current_score = metrics.get("eval_loss") # monitoring 'eval_loss'

        if self.best_score is None or current_score < self.best_score: #compare the evaluation score and reset the patience counter
            self.best_score = current_score
            self.patience_counter = 0
        else:
            self.patience_counter += 1

        if self.patience_counter >= self.early_stopping_patience: # condition if the patient counter exceeds or equal than stop training 
            control.should_training_stop = True


<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Define Training Arguments</p>

In [11]:
training_args = TrainingArguments(
    output_dir="./finetuned_bart_medical", # saved model directory
    per_device_train_batch_size=2, # batch size for each device, set to 2, because of the capacity of the system
    gradient_accumulation_steps=8, # number of steps before performing a backpropagation
    num_train_epochs=2, # number of training epochs, set to 2, due to limited time, this could also take days to train
    logging_dir="./logs", # directory to save logs
    logging_steps=100, # interval to log details like loss
    save_steps=1000, # interval to save the model 
    evaluation_strategy="steps", # evaluate the model every specified number of steps
    save_total_limit=3, # limit the number of saved models to the last 3
    remove_unused_columns=False, # don't remove columns that aren't used by the model from the dataset 
)



<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Initialize  Trainer</p>

In [12]:
trainer = Trainer( # initialize the Trainer instance 
    model=model, # passing the model to be fine tuned
    args=training_args, # passing the training configurations
    train_dataset=train_dataset,  # passing the training dataset
    eval_dataset=val_dataset,  # passing the validation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], # callback function for ealry stopping, with 2 epoch

)


<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Train Model</p>

In [33]:
trainer.train() # training the model


***** Running training *****
  Num examples = 19768
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 2470
  Number of trainable parameters = 139420416


Step,Training Loss,Validation Loss
100,3.171200,0.434019
200,0.444900,0.391237
300,0.413700,0.378523
400,0.415200,0.370626
500,0.401200,0.364687
600,0.371200,0.359647
700,0.386200,0.356392
800,0.380400,0.352636
900,0.386000,0.350098
1000,0.374400,0.346687


***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
Saving model checkpoint to ./finetuned_bart_medical/checkpoint-1000
Configuration saved in ./finetuned_bart_medical/checkpoint-1000/config.json
Model weights saved in ./finetuned_bart_medical/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4943
  Batch size = 8
****

TrainOutput(global_step=2470, training_loss=0.4841944783322724, metrics={'train_runtime': 197147.2414, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.013, 'total_flos': 2.410166272131072e+16, 'train_loss': 0.4841944783322724, 'epoch': 2.0})

<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Save the model</p>

In [ ]:
model.save_pretrained("./finetuned_bart_medical") # save the trained model in the directory
tokenizer.save_pretrained("./finetuned_bart_medical") # save the tokenizer in the directory

<div style="width:100%;height:1px; background-color:black"></div>

<p ><center><u style="font-size: 28px; margin-top: 10px; font-weight: bold">Model Evaluation</u></center></p>

<p style="font-size: 20px">
Model evaluation is an important part of creating machine learning models. It's about testing how good the model is using data it hasn't seen during training, usually called test or validation data. We do this to see if the model's answers are right and understand any mistakes it might make.
</p

<div style="width:100%;height:1px; background-color:black"></div>

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Load the model</p>

In [13]:
model = BartForConditionalGeneration.from_pretrained("./finetuned_bart_medical").to(device)
tokenizer = BartTokenizer.from_pretrained("./finetuned_bart_medical")


loading configuration file ./finetuned_bart_medical/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LAB

<p style="font-size: 23px; margin-top: 10px; font-weight: bold">Loss and Perplexity</p>

<p style="font-size: 20px">
Loss measures the difference between a model's predictions and the actual data. It helps in adjusting the model to make better predictions. By looking at the loss, we can see if the model is improving and make necessary changes if needed.
</p

<p style="font-size: 20px">
    Perplexity checks how good a model is at guessing the next word. For medical chatbots, a lower value means the bot can chat more smoothly and make sense.
</p

<p style="font-size: 20px; margin-top: 10px; font-weight: bold">ROUGE score</p>

<p style="font-size: 20px">
    The ROUGE score looks at how much the predicted text matches the reference text using different measures like precision, recall, and F1-score. It's particularly useful for tasks like summarization to see how much key information the model includes in its output. In the context of medical chatbot, ROUGE can help determine how closely the generated response matches a desired or reference answer, indicating the system's ability to provide accurate and relevant information.
</p>

In [14]:
import numpy as np
from datasets import load_metric

rouge = load_metric("rouge")

def compute_rouge(predictions, references):
    return rouge.compute(predictions=predictions, references=references, use_stemmer=True)

def compute_metrics(eval_prediction: EvalPrediction):
    logits_tuple = eval_prediction.predictions
    labels = eval_prediction.label_ids
    
    # assuming the first tensor in the logits tuple is the actual logits tensor
    logits = logits_tuple[0] if isinstance(logits_tuple, tuple) and len(logits_tuple) > 0 else None

    # convert labels to tensor only if it's a numpy array
    labels_tensor = torch.tensor(labels) if isinstance(labels, np.ndarray) else labels

    # check if we successfully retrieved the logits tensor
    if logits is None:
        raise ValueError("Couldn't retrieve logits tensor from tuple.")

    # convert logits to a tensor
    logits_tensor = torch.tensor(logits)

    # calculate accuracy
    predicted_indices = logits_tensor.argmax(dim=-1)
    correct_predictions = (predicted_indices == labels_tensor).float()
    accuracy = correct_predictions.mean().item()
    
    # calculate ROUGE scores
    predicted_sentences = [tokenizer.decode(pred) for pred in predicted_indices]
    reference_sentences = [tokenizer.decode(label) for label in labels_tensor]
    rouge_scores = compute_rouge(predicted_sentences, reference_sentences)

    # calculate the loss and perplexity
    criterion = nn.CrossEntropyLoss()
    loss = criterion(logits_tensor.view(-1, logits_tensor.shape[-1]), labels_tensor.view(-1))
    perplexity = torch.exp(loss)

    results = {
        "loss": loss.item(),
        "perplexity": perplexity.item(),
        "accuracy": accuracy,
        "rouge-1": {
            "r": rouge_scores['rouge1'].mid.recall,
            "p": rouge_scores['rouge1'].mid.precision,
            "f": rouge_scores['rouge1'].mid.fmeasure
        },
        "rouge-2": {
            "r": rouge_scores['rouge2'].mid.recall,
            "p": rouge_scores['rouge2'].mid.precision,
            "f": rouge_scores['rouge2'].mid.fmeasure
        },
        "rouge-l": {
            "r": rouge_scores['rougeL'].mid.recall,
            "p": rouge_scores['rougeL'].mid.precision,
            "f": rouge_scores['rougeL'].mid.fmeasure
        }
    }

    return results


/home/msc1/anaconda3/envs/Env-7145COMP/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [15]:
subset_val_dataset = val_dataset[:10]


In [16]:
training_args = TrainingArguments(
    per_device_eval_batch_size=1,
    output_dir="./finetuned_bart_medical",
    do_train=False,  # passing false so it will be only used for evaluation
    do_eval=True,
)
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=subset_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
eval_results = trainer.evaluate()
print(eval_results)


***** Running Evaluation *****
  Num examples = 10
  Batch size = 1


Trainer is attempting to log a value of "{'r': 0.9468766505347077, 'p': 0.9548108608335324, 'f': 0.9509627198974746}" of type <class 'dict'> for key "eval/rouge-1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.9114312029118358, 'p': 0.9188992264672378, 'f': 0.9152093838027202}" of type <class 'dict'> for key "eval/rouge-2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.9337740161718167, 'p': 0.9415215814681496, 'f': 0.9377553840850238}" of type <class 'dict'> for key "eval/rouge-l" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.3937792479991913, 'eval_perplexity': 1.4825732707977295, 'eval_accuracy': 0.9208984375, 'eval_rouge-1': {'r': 0.9468766505347077, 'p': 0.9548108608335324, 'f': 0.9509627198974746}, 'eval_rouge-2': {'r': 0.9114312029118358, 'p': 0.9188992264672378, 'f': 0.9152093838027202}, 'eval_rouge-l': {'r': 0.9337740161718167, 'p': 0.9415215814681496, 'f': 0.9377553840850238}, 'eval_runtime': 11.8078, 'eval_samples_per_second': 0.847, 'eval_steps_per_second': 0.847}


In [18]:
eval_results

{'eval_loss': 0.3937792479991913,
 'eval_perplexity': 1.4825732707977295,
 'eval_accuracy': 0.9208984375,
 'eval_rouge-1': {'r': 0.9468766505347077,
  'p': 0.9548108608335324,
  'f': 0.9509627198974746},
 'eval_rouge-2': {'r': 0.9114312029118358,
  'p': 0.9188992264672378,
  'f': 0.9152093838027202},
 'eval_rouge-l': {'r': 0.9337740161718167,
  'p': 0.9415215814681496,
  'f': 0.9377553840850238},
 'eval_runtime': 11.8078,
 'eval_samples_per_second': 0.847,
 'eval_steps_per_second': 0.847}

<p style="font-size: 23px; margin-top: 10px; font-weight: bold"><u>Saving Results to the dataframe</u></p>

<p style="font-size: 20px">
  I am saving the results in the dataframe one by one of each model so i can compare the results in the separate python file (medical_chatbot_eval_metrics.ipynb).
</p>

In [19]:
eval_metrics_results = {
    'model_name': 'Facebook/BART-base',
    'loss': eval_results['eval_loss'],
    'perplexity': eval_results['eval_perplexity'],
    'accuracy': eval_results['eval_accuracy'],
    'rouge-1_r': eval_results['eval_rouge-1']['r'],
    'rouge-1_p': eval_results['eval_rouge-1']['p'],
    'rouge-1_f': eval_results['eval_rouge-1']['f'],
    'rouge-2_r': eval_results['eval_rouge-2']['r'],
    'rouge-2_p': eval_results['eval_rouge-2']['p'],
    'rouge-2_f': eval_results['eval_rouge-2']['f'],
    'rouge-l_r': eval_results['eval_rouge-l']['r'],
    'rouge-l_p': eval_results['eval_rouge-l']['p'],
    'rouge-l_f': eval_results['eval_rouge-l']['f']
}


In [21]:
# eval_metrics_results

In [23]:
import pandas as pd # for dataframe

In [24]:
eval_metrics_results_dataframe = pd.read_csv('eval_metrics_results_dataframe.csv') # load the csv  
eval_metrics_results_dataframe

,model_name,loss,perplexity,accuracy,rouge-1_r,rouge-1_p,rouge-1_f,rouge-2_r,rouge-2_p,rouge-2_f,rouge-l_r,rouge-l_p,rouge-l_f
0,Encoder-Decoder LSTM,0.074382,1.052910,0.990507,0.985153,0.967791,0.976377,0.975947,0.946462,0.960935,0.985153,0.967791,0.976377
1,GPT-2 Medium,10.379869,32204.726562,0.009531,0.497472,0.526547,0.511501,0.176954,0.186899,0.181735,0.388866,0.411154,0.399379


In [25]:
eval_metrics_results_dataframe = eval_metrics_results_dataframe.append(eval_metrics_results, ignore_index=True) # append the record in the dataframe
eval_metrics_results_dataframe.to_csv('eval_metrics_results_dataframe.csv', index=False) # save to the same file
eval_metrics_results_dataframe

,model_name,loss,perplexity,accuracy,rouge-1_r,rouge-1_p,rouge-1_f,rouge-2_r,rouge-2_p,rouge-2_f,rouge-l_r,rouge-l_p,rouge-l_f
0,Encoder-Decoder LSTM,0.074382,1.052910,0.990507,0.985153,0.967791,0.976377,0.975947,0.946462,0.960935,0.985153,0.967791,0.976377
1,GPT-2 Medium,10.379869,32204.726562,0.009531,0.497472,0.526547,0.511501,0.176954,0.186899,0.181735,0.388866,0.411154,0.399379
2,Facebook/BART-base,0.393779,1.482573,0.920898,0.946877,0.954811,0.950963,0.911431,0.918899,0.915209,0.933774,0.941522,0.937755


<p style="font-size: 23px; margin-top: 10px; font-weight: bold"><u>Answer to user queries by using the model</u></p>

In [29]:
def generate_response(user_input):
    # encode the user input and return tensor in PyTorch format
    input_tensor = tokenizer.encode(user_input, return_tensors="pt")
    
    # generate a response from the model
    output_tensor = model.generate(input_tensor, max_length=150, num_beams=5, early_stopping=True)
    
    # decode the response
    response = tokenizer.decode(output_tensor[0], skip_special_tokens=True)
    return response

In [30]:
# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        break
    response = generate_response(user_input)
    print("Bot: ", response)

You:  what is flu?


Bot:  hello and welcome to ask a doctor service. i have reviewed your query and here is my advice.  flu is a viral illness. it is caused by a viral infection. it can be treated with antibiotics.  hope i have answered your query. let me know if i can assist you further.  regards, dr. shinas hussain, general  family physician 


You:  exit
